In [1]:
import pandas as pd 
import numpy as np 

import sys

from pandas                 import DataFrame
from pandas                 import read_csv
from pandas               import get_dummies
from numpy                  import array
from numpy                  import random
from sklearn.linear_model   import LogisticRegression
from sklearn.tree           import DecisionTreeClassifier
from sklearn.ensemble       import RandomForestClassifier
from sklearn.ensemble       import GradientBoostingClassifier
from sklearn.svm            import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors      import KNeighborsClassifier
from sklearn.metrics        import accuracy_score
from sklearn.metrics        import auc
from sklearn.metrics        import roc_auc_scor
from scipy.stats.mstats   import winsorize
from scipy.stats          import pearsonr

In [2]:
test = pd.read_csv('selection campaign 7244.csv', sep= ';')
test.isna().sum()

donorID    0
dtype: int64

In [3]:
train = pd.read_csv('selection campaign 6169.csv', sep= ';')
train

,donorID
0,100001
1,100002
2,100004
3,100008
4,100009
...,...
34883,144686
34884,144687
34885,144689
34886,144690


## Group Project - Predictive analytics

# Data Cleaning

## Donor

In [4]:
donor = pd.read_csv('donors.csv', sep= ';')

donor.loc[donor.province == 'Missing','province'] = 'Antwerp'
donor.loc[donor.province == 'Antwerp','region'] = 'Flanders'
donor.loc[donor.province == 'Antwerp','zipcode'] = 2260.0

donor.gender.fillna('missing', inplace=True)
donor.loc[donor.gender == 'missing','gender'] = 'M'

donor = donor.join(get_dummies(donor["region"], prefix = "region"))
donor = donor.join(get_dummies(donor["province"], prefix = "province"))
donor = donor.join(get_dummies(donor["language"], prefix = "language"))
donor = donor.drop(columns=['province','region','language'])

donor.gender = donor.gender.map({'M':1, 'F':0}).astype(int)
donor.dateOfBirth = pd.to_datetime(donor.dateOfBirth)
#Dropna of the column zipcode
#donor = donor.dropna()
donor.head()


,donorID,zipcode,gender,dateOfBirth,region_Brussels,region_Flanders,region_Wallonia,province_Antwerp,province_Brussels,province_East Flanders,...,province_Hainaut,province_Liege,province_Limburg,province_Luxembourg,province_Namur,province_Walloon Brabant,province_West Flanders,language_EN,language_FR,language_NL
0,100001,1861.0,0,1967-12-24,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,100002,2260.0,1,1952-01-14,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2,100003,1780.0,1,1986-12-23,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,100004,1020.0,0,1952-03-29,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,100005,1020.0,0,1986-06-25,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [5]:
campaign = pd.read_csv('campaigns.csv', sep= ';')
#Remove the "€" sign and change ',' to .
campaign['CostUnit'] = campaign['CostUnit'].replace({'\€':''}, regex = True)
campaign['CostUnit'] = campaign['CostUnit'].replace({'\,':'.'}, regex = True)
#Convert the column in Date format
campaign.date = pd.to_datetime(campaign['date'])
#Object to float format
campaign['CostUnit'] = campaign['CostUnit'].astype(float)
#Object to integer format 
campaign['lettersSent'] = campaign['lettersSent'].replace({'\u202f':''}, regex = True).astype(int)
campaign.head()

,campaignID,date,lettersSent,CostUnit
0,153,2004-12-13,6873,0.17
1,154,2005-01-01,7656,0.30
2,180,2005-01-18,9933,0.20
3,433,2005-01-29,7448,0.59
4,476,2005-12-02,6605,0.51


In [6]:
gift = pd.read_csv('gifts.csv', sep= ';')
gift.date = pd.to_datetime(gift.date)

# Dealing with Outliers:
#print(gift["amount"].min())
#print(gift["amount"].max())
dict = gift["amount"].quantile([.01,.05,.95,.99]).to_dict()
#print(dict)
#Checking for outliers
gift[gift["amount"] >= dict[.99]][["donorID","amount"]]
# Winsorizing to take out outliers 
gift["amount"] = winsorize(array(gift["amount"]), limits=[0.00, 0.01])

gift['month'] = pd.DatetimeIndex(gift['date']).month
gift['days'] = pd.DatetimeIndex(gift['date']).day

day = []
for i in gift['days']:
    if 0<i<6:
        day.append('1-5')
    elif 5<i<11:
         day.append('6-10')
    elif 10<i<16:
         day.append('11-15')
    elif 15<i<21:
         day.append('16-20')
    elif 20<i<26:
         day.append('21-25')
    else:
         day.append('26-31')
     
gift['day'] =  day
gift = gift.drop(['days'], axis=1)

gift = gift.join(get_dummies(gift["day"], prefix = "days"))
gift = gift.join(get_dummies(gift["month"], prefix = "month"))
    
    
gift.head()

,donorID,campaignID,amount,date,month,day,days_1-5,days_11-15,days_16-20,days_21-25,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,100001,NaN,12.39,2004-03-23,3,21-25,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
1,100001,1577.0,12.39,2007-10-11,10,11-15,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,100001,1634.0,6.69,2007-12-28,12,26-31,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,100001,1778.0,24.79,2008-06-14,6,11-15,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
4,100001,1808.0,7.44,2008-07-09,7,6-10,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [8]:
print(len(train.donorID.unique()))
nb_donor_6169 = gift[gift.campaignID==6169]
#len(nb_donor_6169.donorID.unique())

len(gift[gift.campaignID==6169].donorID)
print(1191/34888*100)

34888
3.413781242834212


In [9]:
print(len(test.donorID.unique()))
nb_donor_7244 = gift[gift.campaignID==7244]
len(nb_donor_7244.donorID.unique())

print(361/25645*100)

25645
1.4076818093195556


In [90]:
gift[gift.date.values < campaign_date1.values].pivot_table

Index(['donorID', 'campaignID', 'amount', 'date', 'month', 'day', 'days_1-5',
       'days_11-15', 'days_16-20', 'days_21-25', 'days_26-31', 'days_6-10',
       'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
       'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12'],
      dtype='object')

In [92]:
gift[gift.date.values < campaign_date1.values].pivot_table(['days_1-5',
       'days_11-15', 'days_16-20', 'days_21-25', 'days_26-31', 'days_6-10',
       'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
       'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12'],'donorID', aggfunc='sum')

,days_1-5,days_11-15,days_16-20,days_21-25,days_26-31,days_6-10,month_1,month_10,month_11,month_12,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9
donorID,,,,,,,,,,,,,,,,,,
100001,3,2,0,2,3,2,0,2,0,3,2,1,0,1,2,1,0,0
100002,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
100003,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,1,1
100004,10,3,6,2,1,4,1,2,1,7,4,3,1,0,3,3,1,0
100005,0,0,0,0,1,2,0,2,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144687,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0
144688,0,0,2,0,4,1,0,1,0,0,0,1,1,1,2,1,0,0
144689,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [10]:
campaign_number1 = 6169
campaign_date1 = campaign.date[campaign.campaignID==campaign_number1]
campaign_date1

database1 = gift[gift.date.values < campaign_date1.values]
database1  = pd.merge(database1, train,on='donorID', how='inner')

In [66]:
database1.head()

,donorID,campaignID,amount,date,month,day,days_1-5,days_11-15,days_16-20,days_21-25,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,100001,NaN,12.39,2004-03-23,3,21-25,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
1,100001,1577.0,12.39,2007-10-11,10,11-15,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,100001,1634.0,6.69,2007-12-28,12,26-31,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,100001,1778.0,24.79,2008-06-14,6,11-15,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
4,100001,1808.0,7.44,2008-07-09,7,6-10,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [56]:
days_month_train = database1.pivot_table(['days_1-5',
       'days_11-15', 'days_16-20', 'days_21-25', 'days_26-31', 'days_6-10',
       'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
       'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12'],'donorID', aggfunc='sum')
days_month_train

,days_1-5,days_11-15,days_16-20,days_21-25,days_26-31,days_6-10,month_1,month_10,month_11,month_12,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9
donorID,,,,,,,,,,,,,,,,,,
100001,3,2,0,2,3,2,0,2,0,3,2,1,0,1,2,1,0,0
100002,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
100004,10,3,6,2,1,4,1,2,1,7,4,3,1,0,3,3,1,0
100008,0,1,0,1,1,0,0,1,1,0,0,0,0,0,1,0,0,0
100009,4,3,2,5,3,2,2,2,2,2,1,2,2,0,2,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144686,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0
144687,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0
144689,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0


# TRAIN ISNA()

## ISNA amount

In [47]:
database1_isna = database1[database1.campaignID.isna()]


database1_isna_amount = database1_isna.pivot_table('amount', 'donorID', aggfunc= ['sum', 'count', 'mean', 'last', 'first', 'min', 'max', 'sem', 'std', 'var'])
database1_isna_amount = pd.DataFrame(database1_isna_amount.to_records())
database1_isna_amount = database1_isna_amount.rename(columns={"('sum', 'amount')":"amount_sum", "('count', 'amount')":"amount_count"})
database1_isna_amount = database1_isna_amount.rename(columns={"('mean', 'amount')":"amount_mean", "('last', 'amount')":"amount_last"})
database1_isna_amount = database1_isna_amount.rename(columns={"('first', 'amount')":"amount_first", "('min', 'amount')":"amount_min"})
database1_isna_amount = database1_isna_amount.rename(columns={"('max', 'amount')":"amount_max", "('sem', 'amount')":"amount_sem"})
database1_isna_amount = database1_isna_amount.rename(columns={"('std', 'amount')":"amount_std", "('var', 'amount')":"amount_var"})

database1_isna_amount = database1_isna_amount.add_prefix('isna_')
database1_isna_amount = database1_isna_amount.rename(columns={'isna_donorID':'donorID'})


database1_isna_amount.head()


,donorID,isna_amount_sum,isna_amount_count,isna_amount_mean,isna_amount_last,isna_amount_first,isna_amount_min,isna_amount_max,isna_amount_sem,isna_amount_std,isna_amount_var
0,100001,12.39,1,12.39,12.39,12.39,12.39,12.39,NaN,NaN,NaN
1,100004,24.79,1,24.79,24.79,24.79,24.79,24.79,NaN,NaN,NaN
2,100025,20.00,1,20.00,20.00,20.00,20.00,20.00,NaN,NaN,NaN
3,100026,32.00,1,32.00,32.00,32.00,32.00,32.00,NaN,NaN,NaN
4,100041,24.79,1,24.79,24.79,24.79,24.79,24.79,NaN,NaN,NaN


## ISNA date

In [13]:
date = []
database1_isna_date = database1_isna.pivot_table('date', 'donorID', aggfunc=['first', 'last', 'count'])
database1_isna_date = pd.DataFrame(database1_isna_date.to_records())
database1_isna_date = database1_isna_date.rename(columns={"('first', 'date')":"date_first","('last', 'date')":"date_last", "('count', 'date')":"date_count"})

database1_isna_date['time_diff_first_to_last'] =  (database1_isna_date["date_last"] - database1_isna_date["date_first"]).dt.days
database1_isna_date['time_diff_mean'] = (database1_isna_date['time_diff_first_to_last'] // database1_isna_date["date_count"])

time_since_last = []
for i in database1_isna_date["date_last"]:
    time_since_last.append((campaign_date1 - i).dt.days.values[0])

database1_isna_date['time_since_last'] = time_since_last
database1_isna_date['time_last/diff_mean'] = database1_isna_date['time_since_last'] // database1_isna_date['time_diff_mean']
database1_isna_date['time_last/diff_mean'] = database1_isna_date['time_last/diff_mean'].replace([np.inf, -np.inf], 0)

database1_isna_date = database1_isna_date.add_prefix('isna_')
database1_isna_date = database1_isna_date.rename(columns={'isna_donorID':'donorID'})
database1_isna_date.head()

,donorID,isna_date_first,isna_date_last,isna_date_count,isna_time_diff_first_to_last,isna_time_diff_mean,isna_time_since_last,isna_time_last/diff_mean
0,100001,2004-03-23,2004-03-23,1,0,0,5130,0.0
1,100004,2006-03-30,2006-03-30,1,0,0,4393,0.0
2,100025,2009-05-29,2009-05-29,1,0,0,3237,0.0
3,100026,2011-05-15,2011-05-15,1,0,0,2521,0.0
4,100041,2001-11-14,2001-11-14,1,0,0,5990,0.0


## ISNA merge

In [67]:
database1_isna_merged = pd.merge(database1_isna_amount, database1_isna_date, on='donorID', how='outer')
database1_isna_merged.head()

,donorID,isna_amount_sum,isna_amount_count,isna_amount_mean,isna_amount_last,isna_amount_first,isna_amount_min,isna_amount_max,isna_amount_sem,isna_amount_std,isna_amount_var,isna_date_first,isna_date_last,isna_date_count,isna_time_diff_first_to_last,isna_time_diff_mean,isna_time_since_last,isna_time_last/diff_mean
0,100001,12.39,1,12.39,12.39,12.39,12.39,12.39,NaN,NaN,NaN,2004-03-23,2004-03-23,1,0,0,5130,0.0
1,100004,24.79,1,24.79,24.79,24.79,24.79,24.79,NaN,NaN,NaN,2006-03-30,2006-03-30,1,0,0,4393,0.0
2,100025,20.00,1,20.00,20.00,20.00,20.00,20.00,NaN,NaN,NaN,2009-05-29,2009-05-29,1,0,0,3237,0.0
3,100026,32.00,1,32.00,32.00,32.00,32.00,32.00,NaN,NaN,NaN,2011-05-15,2011-05-15,1,0,0,2521,0.0
4,100041,24.79,1,24.79,24.79,24.79,24.79,24.79,NaN,NaN,NaN,2001-11-14,2001-11-14,1,0,0,5990,0.0


# Tail_3

## Tail amount

In [15]:

database1_tail_3_amount = database1_tail_3 = database1.groupby('donorID').tail(3)


database1_tail_3_amount = database1_tail_3_amount.pivot_table('amount', 'donorID', aggfunc= ['sum', 'count', 'mean', 'last', 'first', 'min', 'max', 'sem', 'std', 'var'])
database1_tail_3_amount = pd.DataFrame(database1_tail_3_amount.to_records())
database1_tail_3_amount = database1_tail_3_amount.rename(columns={"('sum', 'amount')":"amount_sum", "('count', 'amount')":"amount_count"})
database1_tail_3_amount = database1_tail_3_amount.rename(columns={"('mean', 'amount')":"amount_mean", "('last', 'amount')":"amount_last"})
database1_tail_3_amount = database1_tail_3_amount.rename(columns={"('first', 'amount')":"amount_first", "('min', 'amount')":"amount_min"})
database1_tail_3_amount = database1_tail_3_amount.rename(columns={"('max', 'amount')":"amount_max", "('sem', 'amount')":"amount_sem"})
database1_tail_3_amount = database1_tail_3_amount.rename(columns={"('std', 'amount')":"amount_std", "('var', 'amount')":"amount_var"})

database1_tail_3_amount = database1_tail_3_amount.add_prefix('tail_')
database1_tail_3_amount = database1_tail_3_amount.rename(columns={'tail_donorID':'donorID'})


database1_tail_3_amount.head()


,donorID,tail_amount_sum,tail_amount_count,tail_amount_mean,tail_amount_last,tail_amount_first,tail_amount_min,tail_amount_max,tail_amount_sem,tail_amount_std,tail_amount_var
0,100001,30.00,3,10.000000,10.0,10.00,10.00,10.0,0.000000,0.000000,0.000000
1,100002,20.00,1,20.000000,20.0,20.00,20.00,20.0,NaN,NaN,NaN
2,100004,90.00,3,30.000000,30.0,30.00,30.00,30.0,0.000000,0.000000,0.000000
3,100008,124.37,3,41.456667,50.0,49.58,24.79,50.0,8.334215,14.435284,208.377433
4,100009,65.00,3,21.666667,20.0,25.00,20.00,25.0,1.666667,2.886751,8.333333


## Tail date

In [16]:
database1_tail_3_date = database1.groupby('donorID').tail(3)


date = []
database1_tail_3_date = database1_tail_3_date.pivot_table('date', 'donorID', aggfunc=['first', 'last', 'count'])
database1_tail_3_date = pd.DataFrame(database1_tail_3_date.to_records())
database1_tail_3_date= database1_tail_3_date.rename(columns={"('first', 'date')":"date_first","('last', 'date')":"date_last", "('count', 'date')":"date_count"})

database1_tail_3_date['time_diff_first_to_last'] =  (database1_tail_3_date["date_last"] - database1_tail_3_date["date_first"]).dt.days
database1_tail_3_date['time_diff_mean'] = (database1_tail_3_date['time_diff_first_to_last'] // database1_tail_3_date["date_count"])

time_since_last = []
for i in database1_tail_3_date["date_last"]:
    time_since_last.append((campaign_date1 - i).dt.days.values[0])

database1_tail_3_date['time_since_last'] = time_since_last
database1_tail_3_date['time_last/diff_mean'] = database1_tail_3_date['time_since_last'] // database1_tail_3_date['time_diff_mean']
database1_tail_3_date['time_last/diff_mean'] = database1_tail_3_date['time_last/diff_mean'].replace([np.inf, -np.inf], 0)

database1_tail_3_date = database1_tail_3_date.add_prefix('tail_')
database1_tail_3_date = database1_tail_3_date.rename(columns={'tail_donorID':'donorID'})
database1_tail_3_date.head()




,donorID,tail_date_first,tail_date_last,tail_date_count,tail_time_diff_first_to_last,tail_time_diff_mean,tail_time_since_last,tail_time_last/diff_mean
0,100001,2011-02-05,2011-12-30,3,328,109,2292,21.0
1,100002,2015-01-10,2015-01-10,1,0,0,1185,0.0
2,100004,2010-10-01,2011-06-03,3,245,81,2502,30.0
3,100008,2007-06-21,2011-10-28,3,1590,530,2355,4.0
4,100009,2011-08-29,2012-03-25,3,209,69,2206,31.0


In [17]:
database1_tail_3_date['tail_date_last']

0       2011-12-30
1       2015-01-10
2       2011-06-03
3       2011-10-28
4       2012-03-25
           ...    
34864   2015-04-28
34865   2015-11-06
34866   2005-07-22
34867   2014-12-30
34868   2015-12-03
Name: tail_date_last, Length: 34869, dtype: datetime64[ns]

## Tail merge

In [18]:
database1_tail_3_merged = pd.merge(database1_tail_3_date, database1_tail_3_amount, on='donorID', how='outer')
database1_tail_3_merged

,donorID,tail_date_first,tail_date_last,tail_date_count,tail_time_diff_first_to_last,tail_time_diff_mean,tail_time_since_last,tail_time_last/diff_mean,tail_amount_sum,tail_amount_count,tail_amount_mean,tail_amount_last,tail_amount_first,tail_amount_min,tail_amount_max,tail_amount_sem,tail_amount_std,tail_amount_var
0,100001,2011-02-05,2011-12-30,3,328,109,2292,21.0,30.00,3,10.000000,10.00,10.00,10.00,10.00,0.000000,0.000000,0.000000
1,100002,2015-01-10,2015-01-10,1,0,0,1185,0.0,20.00,1,20.000000,20.00,20.00,20.00,20.00,NaN,NaN,NaN
2,100004,2010-10-01,2011-06-03,3,245,81,2502,30.0,90.00,3,30.000000,30.00,30.00,30.00,30.00,0.000000,0.000000,0.000000
3,100008,2007-06-21,2011-10-28,3,1590,530,2355,4.0,124.37,3,41.456667,50.00,49.58,24.79,50.00,8.334215,14.435284,208.377433
4,100009,2011-08-29,2012-03-25,3,209,69,2206,31.0,65.00,3,21.666667,20.00,25.00,20.00,25.00,1.666667,2.886751,8.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34864,144686,2015-04-28,2015-04-28,1,0,0,1077,0.0,50.00,1,50.000000,50.00,50.00,50.00,50.00,NaN,NaN,NaN
34865,144687,2015-11-06,2015-11-06,1,0,0,885,0.0,10.00,1,10.000000,10.00,10.00,10.00,10.00,NaN,NaN,NaN
34866,144689,2005-07-22,2005-07-22,1,0,0,4644,0.0,24.79,1,24.790000,24.79,24.79,24.79,24.79,NaN,NaN,NaN
34867,144690,2014-12-30,2014-12-30,1,0,0,1196,0.0,40.00,1,40.000000,40.00,40.00,40.00,40.00,NaN,NaN,NaN


## NOTNA() amount

In [19]:
database1_notna = database1[database1.campaignID.notna()]


database1_notna_amount = database1_notna.pivot_table('amount', 'donorID', aggfunc= ['sum', 'count', 'mean', 'last', 'first', 'min', 'max', 'sem', 'std', 'var'])
database1_notna_amount = pd.DataFrame(database1_notna_amount.to_records())
database1_notna_amount = database1_notna_amount.rename(columns={"('sum', 'amount')":"amount_sum", "('count', 'amount')":"amount_count"})
database1_notna_amount = database1_notna_amount.rename(columns={"('mean', 'amount')":"amount_mean", "('last', 'amount')":"amount_last"})
database1_notna_amount = database1_notna_amount.rename(columns={"('first', 'amount')":"amount_first", "('min', 'amount')":"amount_min"})
database1_notna_amount = database1_notna_amount.rename(columns={"('max', 'amount')":"amount_max", "('sem', 'amount')":"amount_sem"})
database1_notna_amount = database1_notna_amount.rename(columns={"('std', 'amount')":"amount_std", "('var', 'amount')":"amount_var"})


database1_notna_amount = database1_notna_amount.add_prefix('notna_')
database1_notna_amount = database1_notna_amount.rename(columns={'notna_donorID':'donorID'})
database1_notna_amount.head()


,donorID,notna_amount_sum,notna_amount_count,notna_amount_mean,notna_amount_last,notna_amount_first,notna_amount_min,notna_amount_max,notna_amount_sem,notna_amount_std,notna_amount_var
0,100001,166.10,11,15.100000,10.0,12.39,6.69,30.00,2.476434,8.213404,67.460000
1,100002,20.00,1,20.000000,20.0,20.00,20.00,20.00,NaN,NaN,NaN
2,100004,691.43,25,27.657200,30.0,24.79,24.79,49.58,1.043488,5.217439,27.221671
3,100008,124.37,3,41.456667,50.0,49.58,24.79,50.00,8.334215,14.435284,208.377433
4,100009,326.78,19,17.198947,20.0,12.39,7.44,25.00,1.303565,5.682107,32.286343


## NOTNA DATE

In [20]:
date = []
database1_notna_date = database1_notna.pivot_table('date', 'donorID', aggfunc=['first', 'last', 'count'])
database1_notna_date = pd.DataFrame(database1_notna_date.to_records())
database1_notna_date = database1_notna_date.rename(columns={"('first', 'date')":"date_first","('last', 'date')":"date_last", "('count', 'date')":"date_count"})

database1_notna_date['time_diff_first_to_last'] =  (database1_notna_date["date_last"] - database1_notna_date["date_first"]).dt.days
database1_notna_date['time_diff_mean'] = (database1_notna_date['time_diff_first_to_last'] // database1_notna_date["date_count"])

time_since_last = []
for i in database1_notna_date["date_last"]:
    time_since_last.append((campaign_date1 - i).dt.days.values[0])

database1_notna_date['time_since_last'] = time_since_last
database1_notna_date['time_last/diff_mean'] = database1_notna_date['time_since_last'] // database1_notna_date['time_diff_mean']
database1_notna_date['time_last/diff_mean'] = database1_notna_date['time_last/diff_mean'].replace([np.inf, -np.inf], 0)

database1_notna_date = database1_notna_date.add_prefix('notna_')
database1_notna_date = database1_notna_date.rename(columns={'notna_donorID':'donorID'})
database1_notna_date.head()

,donorID,notna_date_first,notna_date_last,notna_date_count,notna_time_diff_first_to_last,notna_time_diff_mean,notna_time_since_last,notna_time_last/diff_mean
0,100001,2007-10-11,2011-12-30,11,1541,140,2292,16.0
1,100002,2015-01-10,2015-01-10,1,0,0,1185,0.0
2,100004,2005-07-02,2011-06-03,25,2162,86,2502,29.0
3,100008,2007-06-21,2011-10-28,3,1590,530,2355,4.0
4,100009,2006-01-17,2012-03-25,19,2259,118,2206,18.0


## Notna merge

In [21]:
database1_notna_merged = pd.merge(database1_notna_amount, database1_notna_date, on='donorID', how='outer')
database1_notna_merged

,donorID,notna_amount_sum,notna_amount_count,notna_amount_mean,notna_amount_last,notna_amount_first,notna_amount_min,notna_amount_max,notna_amount_sem,notna_amount_std,notna_amount_var,notna_date_first,notna_date_last,notna_date_count,notna_time_diff_first_to_last,notna_time_diff_mean,notna_time_since_last,notna_time_last/diff_mean
0,100001,166.10,11,15.100000,10.00,12.39,6.69,30.00,2.476434,8.213404,67.460000,2007-10-11,2011-12-30,11,1541,140,2292,16.0
1,100002,20.00,1,20.000000,20.00,20.00,20.00,20.00,NaN,NaN,NaN,2015-01-10,2015-01-10,1,0,0,1185,0.0
2,100004,691.43,25,27.657200,30.00,24.79,24.79,49.58,1.043488,5.217439,27.221671,2005-07-02,2011-06-03,25,2162,86,2502,29.0
3,100008,124.37,3,41.456667,50.00,49.58,24.79,50.00,8.334215,14.435284,208.377433,2007-06-21,2011-10-28,3,1590,530,2355,4.0
4,100009,326.78,19,17.198947,20.00,12.39,7.44,25.00,1.303565,5.682107,32.286343,2006-01-17,2012-03-25,19,2259,118,2206,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33772,144686,50.00,1,50.000000,50.00,50.00,50.00,50.00,NaN,NaN,NaN,2015-04-28,2015-04-28,1,0,0,1077,0.0
33773,144687,10.00,1,10.000000,10.00,10.00,10.00,10.00,NaN,NaN,NaN,2015-11-06,2015-11-06,1,0,0,885,0.0
33774,144689,24.79,1,24.790000,24.79,24.79,24.79,24.79,NaN,NaN,NaN,2005-07-22,2005-07-22,1,0,0,4644,0.0
33775,144690,40.00,1,40.000000,40.00,40.00,40.00,40.00,NaN,NaN,NaN,2014-12-30,2014-12-30,1,0,0,1196,0.0


# TRAIN general merge isna + notna + tail3

In [78]:
database1_merged = pd.merge(database1_notna_merged, database1_isna_merged, on='donorID', how='outer')
database1_merged.fillna(0, inplace = True)
donor['age'] = ((campaign_date1.values[0] - donor.dateOfBirth)/365).dt.days
database1_merged = pd.merge(donor, database1_merged, on= 'donorID', how='right') 
#drop all date for the ML
database1_merged = pd.merge(database1_merged, database1_tail_3_merged, on= 'donorID', how='outer')
database1_merged = database1_merged.drop(columns=['dateOfBirth','tail_date_first','tail_date_last','notna_date_first','notna_date_last','isna_date_first','isna_date_last'])
database1_merged = pd.merge(database1_merged, days_month_train, on='donorID', how='outer')
database1_merged.fillna(0, inplace = True)
database1_merged.head()

,donorID,zipcode,gender,region_Brussels,region_Flanders,region_Wallonia,province_Antwerp,province_Brussels,province_East Flanders,province_Flemish Brabant,...,month_11,month_12,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9
0,100001,1861.0,0,0,1,0,0,0,0,1,...,0,3,2,1,0,1,2,1,0,0
1,100002,2260.0,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,100004,1020.0,0,1,0,0,0,1,0,0,...,1,7,4,3,1,0,3,3,1,0
3,100008,1780.0,1,0,1,0,0,0,0,1,...,1,0,0,0,0,0,1,0,0,0
4,100009,1020.0,1,1,0,0,0,1,0,0,...,2,2,1,2,2,0,2,1,2,1


In [23]:
train['donorID'].values
gift[gift.campaignID==6169].donorID.values
len(database1_merged.donorID)

34869

In [79]:
present = []
for i in database1_merged.donorID:
    if i in gift[gift.campaignID==6169].donorID.values:
        present.append(1)
    else:
        present.append(0)
        
database1_merged['target'] = present
database1_merged

,donorID,zipcode,gender,region_Brussels,region_Flanders,region_Wallonia,province_Antwerp,province_Brussels,province_East Flanders,province_Flemish Brabant,...,month_12,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,target
0,100001,1861.0,0,0,1,0,0,0,0,1,...,3,2,1,0,1,2,1,0,0,0
1,100002,2260.0,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,100004,1020.0,0,1,0,0,0,1,0,0,...,7,4,3,1,0,3,3,1,0,0
3,100008,1780.0,1,0,1,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,100009,1020.0,1,1,0,0,0,1,0,0,...,2,1,2,2,0,2,1,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34864,144552,3471.0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
34865,144605,9000.0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
34866,144621,1703.0,1,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,1
34867,144649,3631.0,1,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


# TEST SET 

In [356]:
challenge = pd.read_csv('selection campaign 7662 (without gifts).csv', sep=';')
challenge.count()

donorID    26522
dtype: int64

In [357]:
campaign.date[campaign.campaignID==campaign_number2] + pd.DateOffset(years=1)

366   2020-12-15
Name: date, dtype: datetime64[ns]

In [358]:
campaign.sort_values('date').tail(1)

,campaignID,date,lettersSent,CostUnit
366,7536,2019-12-15,37187,0.35


In [359]:
#Extract date of the campaign
campaign_number2 = 7536
campaign_date2 = campaign.date[campaign.campaignID==campaign_number2] + pd.DateOffset(years=1)
#Delete all donation after the date campaign to have data history 
database2 = gift[gift.date.values < campaign_date2.values]
database2  = pd.merge(database2, challenge,on='donorID', how='right')  #chalenge or test

In [360]:
campaign_date2

366   2020-12-15
Name: date, dtype: datetime64[ns]

In [361]:
days_month_test = database2.pivot_table(['days_1-5',
       'days_11-15', 'days_16-20', 'days_21-25', 'days_26-31', 'days_6-10',
       'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
       'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12'],'donorID', aggfunc='sum')
days_month_test

,days_1-5,days_11-15,days_16-20,days_21-25,days_26-31,days_6-10,month_1,month_10,month_11,month_12,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9
donorID,,,,,,,,,,,,,,,,,,
100001,3.0,2.0,0.0,2.0,3.0,2.0,0.0,2.0,0.0,3.0,2.0,1.0,0.0,1.0,2.0,1.0,0.0,0.0
100003,1.0,2.0,2.0,3.0,1.0,3.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,1.0,2.0,2.0,1.0,2.0
100004,10.0,3.0,6.0,2.0,1.0,4.0,1.0,2.0,1.0,7.0,4.0,3.0,1.0,0.0,3.0,3.0,1.0,0.0
100006,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
100009,4.0,3.0,2.0,5.0,3.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,0.0,2.0,1.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144687,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
144688,0.0,0.0,2.0,0.0,4.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,2.0,1.0,0.0,0.0
144689,1.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0


In [362]:
database2[['donorID']].sort_values('donorID').count()
challenge.head().sort_values('donorID').count()

donorID    5
dtype: int64

## ISNA amount

In [363]:
###########################
##DATA ISNA donation###
###########################
database2_isna = database2[database2.campaignID.isna()]

#Pivot table with amount 
database2_isna_amount = database2_isna.pivot_table('amount', 'donorID', aggfunc= ['sum', 'count', 'mean', 'last', 'first', 'min', 'max', 'sem', 'std', 'var'])
database2_isna_amount = pd.DataFrame(database2_isna_amount.to_records())
database2_isna_amount = database2_isna_amount.rename(columns={"('sum', 'amount')":"isna_amount_sum", "('count', 'amount')":"isna_amount_count"})
database2_isna_amount = database2_isna_amount.rename(columns={"('mean', 'amount')":"isna_amount_mean", "('last', 'amount')":"isna_amount_last"})
database2_isna_amount = database2_isna_amount.rename(columns={"('first', 'amount')":"isna_amount_first", "('min', 'amount')":"isna_amount_min"})
database2_isna_amount = database2_isna_amount.rename(columns={"('max', 'amount')":"isna_amount_max", "('sem', 'amount')":"isna_amount_sem"})
database2_isna_amount = database2_isna_amount.rename(columns={"('std', 'amount')":"isna_amount_std", "('var', 'amount')":"isna_amount_var"})

#database2_isna_amount = database2_isna_amount.add_prefix('isna_')
#database2_isna_amount = database2_isna_amount.rename(columns={'isna_donorID':'donorID'})
#database2_isna_amount = database2_isna_merged.drop(columns={'index'})
database2_isna_amount

,donorID,isna_amount_sum,isna_amount_count,isna_amount_mean,isna_amount_last,isna_amount_first,isna_amount_min,isna_amount_max,isna_amount_sem,isna_amount_std,isna_amount_var
0,100001,12.39,1,12.39,12.39,12.39,12.39,12.39,NaN,NaN,NaN
1,100004,24.79,1,24.79,24.79,24.79,24.79,24.79,NaN,NaN,NaN
2,100020,24.79,1,24.79,24.79,24.79,24.79,24.79,NaN,NaN,NaN
3,100026,32.00,1,32.00,32.00,32.00,32.00,32.00,NaN,NaN,NaN
4,100041,24.79,1,24.79,24.79,24.79,24.79,24.79,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3419,144666,32.25,1,32.25,32.25,32.25,32.25,32.25,NaN,NaN,NaN
3420,144667,49.58,1,49.58,49.58,49.58,49.58,49.58,NaN,NaN,NaN
3421,144668,20.00,1,20.00,20.00,20.00,20.00,20.00,NaN,NaN,NaN
3422,144683,50.00,1,50.00,50.00,50.00,50.00,50.00,NaN,NaN,NaN


## ISNA date

In [364]:
#Pivot table with date 
date = []
database2_isna_date = database2_isna.pivot_table('date', 'donorID', aggfunc=['first', 'last', 'count'])
database2_isna_date = pd.DataFrame(database2_isna_date.to_records())
database2_isna_date = database2_isna_date.rename(columns={"('first', 'date')":"date_first","('last', 'date')":"date_last", "('count', 'date')":"date_count"})

database2_isna_date['time_diff_first_to_last'] =  (database2_isna_date["date_last"] - database2_isna_date["date_first"]).dt.days
database2_isna_date['time_diff_mean'] = (database2_isna_date['time_diff_first_to_last'] // database2_isna_date["date_count"])


time_since_last = []
for i in database2_isna_date["date_last"]:
    time_since_last.append((campaign_date2 - i).dt.days.values[0])

database2_isna_date['time_since_last'] = time_since_last
database2_isna_date['time_last/diff_mean'] = database2_isna_date['time_since_last'] // database2_isna_date['time_diff_mean']
database2_isna_date['time_last/diff_mean'] = database2_isna_date['time_last/diff_mean'].replace([np.inf, -np.inf], 0)

database2_isna_date = database2_isna_date.add_prefix('isna_')
database2_isna_date = database2_isna_date.rename(columns={'isna_donorID':'donorID'})
database2_isna_date.head()

,donorID,isna_date_first,isna_date_last,isna_date_count,isna_time_diff_first_to_last,isna_time_diff_mean,isna_time_since_last,isna_time_last/diff_mean
0,100001,2004-03-23,2004-03-23,1,0.0,0.0,6111.0,0.0
1,100004,2006-03-30,2006-03-30,1,0.0,0.0,5374.0,0.0
2,100020,2001-12-05,2001-12-05,1,0.0,0.0,6950.0,0.0
3,100026,2011-05-15,2011-05-15,1,0.0,0.0,3502.0,0.0
4,100041,2001-11-14,2001-11-14,1,0.0,0.0,6971.0,0.0


## Merge isna 

In [365]:
#Merge the 2 pivot table for isna()
database2_isna_merged = pd.merge(database2_isna_amount, database2_isna_date, on='donorID', how='outer')
database2_isna_merged = database2_isna_merged.drop(columns='index')
database2_isna_merged.head()

KeyError: "['index'] not found in axis"

## TAIL amount

In [ ]:
database2_tail_3_amount = database2_tail_3 = database2.groupby('donorID').tail(3)


database2_tail_3_amount = database2_tail_3_amount.pivot_table('amount', 'donorID', aggfunc= ['sum', 'count', 'mean', 'last', 'first', 'min', 'max', 'sem', 'std', 'var'])
database2_tail_3_amount = pd.DataFrame(database2_tail_3_amount.to_records())
database2_tail_3_amount = database2_tail_3_amount.rename(columns={"('sum', 'amount')":"amount_sum", "('count', 'amount')":"amount_count"})
database2_tail_3_amount = database2_tail_3_amount.rename(columns={"('mean', 'amount')":"amount_mean", "('last', 'amount')":"amount_last"})
database2_tail_3_amount = database2_tail_3_amount.rename(columns={"('first', 'amount')":"amount_first", "('min', 'amount')":"amount_min"})
database2_tail_3_amount = database2_tail_3_amount.rename(columns={"('max', 'amount')":"amount_max", "('sem', 'amount')":"amount_sem"})
database2_tail_3_amount = database2_tail_3_amount.rename(columns={"('std', 'amount')":"amount_std", "('var', 'amount')":"amount_var"})

database2_tail_3_amount = database2_tail_3_amount.add_prefix('tail_')
database2_tail_3_amount = database2_tail_3_amount.rename(columns={'tail_donorID':'donorID'})


database2_tail_3_amount.head()

## Tail Date

In [ ]:
database2_tail_3_date = database2.groupby('donorID').tail(3)


date = []
database2_tail_3_date = database2_tail_3_date.pivot_table('date', 'donorID', aggfunc=['first', 'last', 'count'])
database2_tail_3_date = pd.DataFrame(database2_tail_3_date.to_records())
database2_tail_3_date= database2_tail_3_date.rename(columns={"('first', 'date')":"date_first","('last', 'date')":"date_last", "('count', 'date')":"date_count"})

database2_tail_3_date['time_diff_first_to_last'] =  (database2_tail_3_date["date_last"] - database2_tail_3_date["date_first"]).dt.days
database2_tail_3_date['time_diff_mean'] = (database2_tail_3_date['time_diff_first_to_last'] // database2_tail_3_date["date_count"])

time_since_last = []
for i in database2_tail_3_date["date_last"]:
    time_since_last.append((campaign_date1 - i).dt.days.values[0])

database2_tail_3_date['time_since_last'] = time_since_last
database2_tail_3_date['time_last/diff_mean'] = database2_tail_3_date['time_since_last'] // database2_tail_3_date['time_diff_mean']
database2_tail_3_date['time_last/diff_mean'] = database2_tail_3_date['time_last/diff_mean'].replace([np.inf, -np.inf], 0)

database2_tail_3_date = database2_tail_3_date.add_prefix('tail_')
database2_tail_3_date = database2_tail_3_date.rename(columns={'tail_donorID':'donorID'})
database2_tail_3_date.head()

## MERGE TAIL

In [ ]:
database2_tail_3_merged = pd.merge(database2_tail_3_date, database2_tail_3_amount, on='donorID', how='outer')
database2_tail_3_merged

## NOTNA amount

In [366]:
##############################
##DATA NOTNA donation###
##############################


database2_notna = database2[database2.campaignID.notna()]

#Pivot table with amount 
database2_notna_amount = database2_notna.pivot_table('amount', 'donorID', aggfunc= ['sum', 'count', 'mean', 'last', 'first', 'min', 'max', 'sem', 'std', 'var'])
database2_notna_amount = pd.DataFrame(database2_notna_amount.to_records())
database2_notna_amount = database2_notna_amount.rename(columns={"('sum', 'amount')":"amount_sum", "('count', 'amount')":"amount_count"})
database2_notna_amount = database2_notna_amount.rename(columns={"('mean', 'amount')":"amount_mean", "('last', 'amount')":"amount_last"})
database2_notna_amount = database2_notna_amount.rename(columns={"('first', 'amount')":"amount_first", "('min', 'amount')":"amount_min"})
database2_notna_amount = database2_notna_amount.rename(columns={"('max', 'amount')":"amount_max", "('sem', 'amount')":"amount_sem"})
database2_notna_amount = database2_notna_amount.rename(columns={"('std', 'amount')":"amount_std", "('var', 'amount')":"amount_var"})


database2_notna_amount = database2_notna_amount.add_prefix('notna_')
database2_notna_amount = database2_notna_amount.rename(columns={'notna_donorID':'donorID'})
database2_notna_amount


,donorID,notna_amount_sum,notna_amount_count,notna_amount_mean,notna_amount_last,notna_amount_first,notna_amount_min,notna_amount_max,notna_amount_sem,notna_amount_std,notna_amount_var
0,100001,166.10,11,15.100000,10.00,12.39,6.69,30.00,2.476434,8.213404,67.460000
1,100003,46.00,12,3.833333,4.00,5.00,3.00,5.00,0.166667,0.577350,0.333333
2,100004,691.43,25,27.657200,30.00,24.79,24.79,49.58,1.043488,5.217439,27.221671
3,100006,5.00,1,5.000000,5.00,5.00,5.00,5.00,NaN,NaN,NaN
4,100009,326.78,19,17.198947,20.00,12.39,7.44,25.00,1.303565,5.682107,32.286343
...,...,...,...,...,...,...,...,...,...,...,...
25792,144687,10.00,1,10.000000,10.00,10.00,10.00,10.00,NaN,NaN,NaN
25793,144688,43.56,6,7.260000,15.00,7.44,2.48,15.00,1.722905,4.220237,17.810400
25794,144689,348.74,4,87.185000,123.95,24.79,24.79,123.95,21.550811,43.101623,1857.749900
25795,144690,40.00,1,40.000000,40.00,40.00,40.00,40.00,NaN,NaN,NaN


## NOTNA date

In [367]:
#Pivot table with date 
date = []
database2_notna_date = database2_notna.pivot_table('date', 'donorID', aggfunc=['first', 'last', 'count'])
database2_notna_date = pd.DataFrame(database2_notna_date.to_records())
database2_notna_date = database2_notna_date.rename(columns={"('first', 'date')":"date_first","('last', 'date')":"date_last", "('count', 'date')":"date_count"})

database2_notna_date['time_diff_first_to_last'] =  (database2_notna_date["date_last"] - database2_notna_date["date_first"]).dt.days
database2_notna_date['time_diff_mean'] = (database2_notna_date['time_diff_first_to_last'] // database2_notna_date["date_count"])

time_since_last = []
for i in database2_notna_date["date_last"]:
    time_since_last.append((campaign_date2 - i).dt.days.values[0])


database2_notna_date['time_since_last'] = time_since_last
database2_notna_date['time_last/diff_mean'] = database2_notna_date['time_since_last'] // database2_notna_date['time_diff_mean']
database2_notna_date['time_last/diff_mean'] = database2_notna_date['time_last/diff_mean'].replace([np.inf, -np.inf], 0)

database2_notna_date = database2_notna_date.add_prefix('notna_')
database2_notna_date = database2_notna_date.rename(columns={'notna_donorID':'donorID'})
database2_notna_date

,donorID,notna_date_first,notna_date_last,notna_date_count,notna_time_diff_first_to_last,notna_time_diff_mean,notna_time_since_last,notna_time_last/diff_mean
0,100001,2007-10-11,2011-12-30,11,1541,140,3273,23.0
1,100003,2017-07-27,2019-06-17,12,690,57,547,9.0
2,100004,2005-07-02,2011-06-03,25,2162,86,3483,40.0
3,100006,2017-08-15,2017-08-15,1,0,0,1218,0.0
4,100009,2006-01-17,2012-03-25,19,2259,118,3187,27.0
...,...,...,...,...,...,...,...,...
25792,144687,2015-11-06,2015-11-06,1,0,0,1866,0.0
25793,144688,2008-04-16,2017-07-27,6,3389,564,1237,2.0
25794,144689,2005-07-22,2019-12-06,4,5250,1312,375,0.0
25795,144690,2014-12-30,2014-12-30,1,0,0,2177,0.0


## Merge notna 

In [368]:
#Merge the 2 pivot table for notna 
database2_notna_merged = pd.merge(database2_notna_amount, database2_notna_date, on='donorID', how='outer')
database2_notna_merged.head()

,donorID,notna_amount_sum,notna_amount_count,notna_amount_mean,notna_amount_last,notna_amount_first,notna_amount_min,notna_amount_max,notna_amount_sem,notna_amount_std,notna_amount_var,notna_date_first,notna_date_last,notna_date_count,notna_time_diff_first_to_last,notna_time_diff_mean,notna_time_since_last,notna_time_last/diff_mean
0,100001,166.10,11,15.100000,10.0,12.39,6.69,30.00,2.476434,8.213404,67.460000,2007-10-11,2011-12-30,11,1541,140,3273,23.0
1,100003,46.00,12,3.833333,4.0,5.00,3.00,5.00,0.166667,0.577350,0.333333,2017-07-27,2019-06-17,12,690,57,547,9.0
2,100004,691.43,25,27.657200,30.0,24.79,24.79,49.58,1.043488,5.217439,27.221671,2005-07-02,2011-06-03,25,2162,86,3483,40.0
3,100006,5.00,1,5.000000,5.0,5.00,5.00,5.00,NaN,NaN,NaN,2017-08-15,2017-08-15,1,0,0,1218,0.0
4,100009,326.78,19,17.198947,20.0,12.39,7.44,25.00,1.303565,5.682107,32.286343,2006-01-17,2012-03-25,19,2259,118,3187,27.0


## TEST General merge isna + notna + tail3 + days_month_test

In [369]:
# MERGE of notna and isna table
database2_merged = pd.merge(database2_notna_merged, database2_isna_merged, on='donorID', how='outer')
database2_merged.fillna(0, inplace = True)
donor['age'] = ((campaign_date2.values[0] - donor.dateOfBirth)/365).dt.days
database2_merged = pd.merge(donor, database2_merged, on= 'donorID', how='right') 
#drop all date for the ML
database2_merged = pd.merge(database2_merged, database2_tail_3_merged, on= 'donorID', how='outer')
database2_merged = database2_merged.drop(columns=['dateOfBirth','notna_date_first','tail_date_last','tail_date_first','notna_date_last','isna_date_first','isna_date_last'])
database2_merged = pd.merge(database2_merged, days_month_test, on= 'donorID', how='outer')
database2_merged.fillna(0, inplace = True)
database2_merged.head()

,donorID,zipcode,gender,region_Brussels,region_Flanders,region_Wallonia,province_Antwerp,province_Brussels,province_East Flanders,province_Flemish Brabant,...,month_11,month_12,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9
0,100001,1861.0,0,0,1,0,0,0,0,1,...,0.0,3.0,2.0,1.0,0.0,1.0,2.0,1.0,0.0,0.0
1,100003,1780.0,1,0,1,0,0,0,0,1,...,0.0,0.0,0.0,2.0,0.0,1.0,2.0,2.0,1.0,2.0
2,100004,1020.0,0,1,0,0,0,1,0,0,...,1.0,7.0,4.0,3.0,1.0,0.0,3.0,3.0,1.0,0.0
3,100006,1090.0,1,1,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,100009,1020.0,1,1,0,0,0,1,0,0,...,2.0,2.0,1.0,2.0,2.0,0.0,2.0,1.0,2.0,1.0


In [240]:
# Add the target columns
present = []
for i in database2_merged.donorID:
    if i in gift[gift.campaignID==7244].donorID.values:
        present.append(1)
    else:
        present.append(0)
        
database2_merged['target'] = present
database2_merged.head()


,donorID,zipcode,gender,region_Brussels,region_Flanders,region_Wallonia,province_Antwerp,province_Brussels,province_East Flanders,province_Flemish Brabant,...,month_12,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,target
0,100001,1861.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,3.0,2.0,1.0,0.0,1.0,2.0,1.0,0.0,0.0,0
1,100002,2260.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,100008,1780.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
3,100009,1020.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,2.0,1.0,2.0,2.0,0.0,2.0,1.0,2.0,1.0,0
4,100010,1020.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [241]:
#moyenne amount when campaignID is n

In [38]:
len(database2_merged.donorID.unique())

26890

In [370]:
testSet = database2_merged
trainingSet = database1_merged


In [371]:
# define features and target
trainingSet_features = trainingSet.columns[0:-1]
testSet_features = testSet.columns #testSet.columns[0:-1]
target = 'target'

In [372]:
tree         = DecisionTreeClassifier()
logistic     = LogisticRegression(solver = "lbfgs", max_iter = 500)
randomForest = RandomForestClassifier(n_estimators = 1000)
boostedTree  = GradientBoostingClassifier()
svm          = SVC(gamma = "scale", probability = True)
neuralNet    = MLPClassifier()
neighbors    = KNeighborsClassifier()

models = {"tree"         :tree,
          "logistic"     :logistic,
          "randomForest" :randomForest,
          "boostedTree"  :boostedTree,
          #"svm"          :svm,
          #"neuralNet"    :neuralNet,
          "neighbors"    :neighbors
         }

In [373]:
for model in models:
    models[model].fit(trainingSet[trainingSet_features],trainingSet[target])
    print(f"{model} has been trained successfully")

tree has been trained successfully


/Users/theaubruno/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


logistic has been trained successfully
randomForest has been trained successfully
boostedTree has been trained successfully
neighbors has been trained successfully


In [246]:

performances = {}

for model in models:
    predictions   = models[model].predict(testSet[testSet_features])
    probabilities = DataFrame(models[model].predict_proba(testSet[testSet_features]))[1]
    accuracy      = accuracy_score(testSet[target],predictions)
    auc           = roc_auc_score(array(testSet[target]),array(probabilities))
    
    performances[model] = {"Accuracy":accuracy,"AUC":auc}

pd.DataFrame(performances)

,tree,logistic,randomForest,boostedTree,neighbors
Accuracy,0.928040,0.986575,0.986352,0.984158,0.986240
AUC,0.505855,0.564550,0.727456,0.714059,0.528095


In [44]:
from ieseg import partition
from ieseg import roc
from ieseg import lift
from ieseg import cumulativeResponse
from ieseg import cumulativeGains

In [247]:
selectedFeatures = []
target = target

trainingSet_columns = trainingSet_features

for column in trainingSet_columns:
        (pearson,pvalue) = pearsonr(trainingSet[column],trainingSet[target])
        print(f"{column} - p-value = {pvalue} - selected : {1 if pvalue < 0.005 else 0}")
        if pvalue < 0.005:
            selectedFeatures.append(column)

donorID - p-value = 0.1259612097316679 - selected : 0
zipcode - p-value = 0.12299634802580232 - selected : 0
gender - p-value = 5.749552717226543e-74 - selected : 1
region_Brussels - p-value = 0.09614236593950763 - selected : 0
region_Flanders - p-value = 0.0558670405330267 - selected : 0
region_Wallonia - p-value = 0.27916876053106265 - selected : 0
province_Antwerp - p-value = 0.380528681110385 - selected : 0
province_Brussels - p-value = 0.09614236593950763 - selected : 0
province_East Flanders - p-value = 0.06127473852107413 - selected : 0
province_Flemish Brabant - p-value = 0.5627784975941211 - selected : 0
province_Hainaut - p-value = 0.17645913862408844 - selected : 0
province_Liege - p-value = 0.5078090467362996 - selected : 0
province_Limburg - p-value = 0.9896366560725678 - selected : 0
province_Luxembourg - p-value = 0.9639155681365967 - selected : 0
province_Namur - p-value = 0.9210086637615648 - selected : 0
province_Walloon Brabant - p-value = 0.07263535849275005 - selec

In [248]:
print(len(trainingSet_features))
print(len(selectedFeatures))
selectedFeatures

84
20


['gender',
 'age',
 'notna_amount_sum',
 'notna_amount_count',
 'notna_date_count',
 'notna_time_diff_first_to_last',
 'notna_time_diff_mean',
 'notna_time_since_last',
 'tail_date_count',
 'tail_time_diff_first_to_last',
 'tail_time_diff_mean',
 'tail_time_since_last',
 'tail_time_last/diff_mean',
 'tail_amount_sum',
 'tail_amount_count',
 'days_11-15',
 'days_21-25',
 'month_2',
 'month_3',
 'month_6']

In [374]:
newtestset = testSet
newtestset['proba_model_prediction'] = randomForest.predict_proba(testSet[testSet_features])[:,0]
testsetforprofessor = newtestset[['donorID','proba_model_prediction']]
testsetforprofessor

testsetforprofessor.to_csv('group1randomforest(thomas&theau).csv',index=True)

In [375]:
testsetforprofessor

,donorID,proba_model_prediction
0,100001,0.993
1,100003,0.805
2,100004,0.998
3,100006,0.847
4,100009,0.982
...,...,...
26517,144314,0.903
26518,144333,0.950
26519,144404,0.959
26520,144605,0.990


In [377]:
pd.merge(testsetforprofessor, challenge, on='donorID', how='right')

,donorID,proba_model_prediction
0,134242,0.999
1,126503,0.998
2,104392,1.000
3,105905,0.864
4,120506,1.000
...,...,...
26517,114719,0.999
26518,134824,0.995
26519,143259,0.993
26520,123936,0.926


In [392]:
m = challenge['donorID'].sort_values().unique() == testsetforprofessor['donorID'].sort_values().unique()

array([100001, 100003, 100004, ..., 144689, 144690, 144691])